<a href="https://colab.research.google.com/github/SanjilMahajani/SubtitleGenerator/blob/main/Video_Subtitle_creator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install git+https://github.com/openai/whisper.git
!pip install ffmpeg-python
!pip install torch torchvision torchaudio --extra-index-url https://download.pytorch.org/whl/cu117



In [ ]:
import ffmpeg
import whisper
import torch

# Check if GPU is available and select the device
device = "cuda" if torch.cuda.is_available() else "cpu"

# Load the Whisper model on the selected device
model = whisper.load_model("medium", device=device)  # Use 'base', 'small', 'medium', or 'large' based on your requirement


In [ ]:
import os
import whisper
import ffmpeg

# Check if GPU is available and select the device
device = "cuda" if torch.cuda.is_available() else "cpu"

# Function to extract audio from video
def extract_audio_from_video(video_path, audio_path="your location.wav"):
    try:
        # Use ffmpeg to extract audio
        ffmpeg.input(video_path).output(audio_path).run(overwrite_output=True, quiet=True)
        print(f"Audio extracted to {audio_path}")
    except Exception as e:
        print(f"Error extracting audio: {e}")

# Function to convert seconds to SRT time format
def convert_seconds_to_srt_time(seconds):
    hours = int(seconds // 3600)
    minutes = int((seconds % 3600) // 60)
    seconds = int(seconds % 60)
    milliseconds = int((seconds % 1) * 1000)
    return f"{hours:02}:{minutes:02}:{seconds:02},{milliseconds:03}"

# Function to transcribe audio to English text using Whisper with timestamps
def transcribe_audio_to_english_with_timestamps(model, audio_path):
    try:
        # Transcribe audio using Whisper with translation to English and timestamps
        result = model.transcribe(audio_path, task="translate", condition_on_previous_text=False)
        segments = result['segments']
        return segments
    except Exception as e:
        print(f"Error transcribing audio: {e}")
        return None

# Function to create SRT file from Whisper transcription segments
def create_srt_from_segments(segments, output_srt_file):
    with open(output_srt_file, 'w') as srt_file:
        for index, segment in enumerate(segments):
            start_time = segment['start']
            end_time = segment['end']
            text = segment['text']
            start_time_str = convert_seconds_to_srt_time(start_time)
            end_time_str = convert_seconds_to_srt_time(end_time)
            srt_file.write(f"{index + 1}\n")
            srt_file.write(f"{start_time_str} --> {end_time_str}\n")
            srt_file.write(f"{text.strip()}\n\n")

    print(f"Subtitle file '{output_srt_file}' created successfully.")

# Main function to convert video to English subtitles
def video_to_english_subtitles(video_path):
    audio_path = "your location.wav"
    extract_audio_from_video(video_path, audio_path)

    segments = transcribe_audio_to_english_with_timestamps(model, audio_path)
    if segments:
        create_srt_from_segments(segments, "your location.srt")
    else:
        print("Audio transcription and translation failed.")


video_path = "your location.mp4"  # Replace with your video file path
video_to_english_subtitles(video_path)
